# Cygnus X-3 analysis notebook

## Prerequisities: in order to run this notebook you need the AGILE data.

Hypothesis: there is a probable flare from Cygnus X-3. Check http://www.astronomerstelegram.org/?read=13458

In [ ]:
from agilepy.api.AGAnalysis import AGAnalysis

In [ ]:
#%matplotlib notebook
%matplotlib widget

## Creating a configuration file

In [ ]:
confFilePath = "./agilepy_conf.yaml"

In [ ]:
AGAnalysis.getConfiguration(
    confFilePath = confFilePath,
    userName = "bulgarelli",
    sourceName = "CYGX3",
    tmin = 58884.0,
    tmax = 58886.0,
    timetype = "MJD",
    glon = 79.8,
    glat = 0.7,
    outputDir = "$HOME/agilepy_analysis",
    verboselvl = 1
)

## Obtaining the AGAnalysis object

In [ ]:
ag = AGAnalysis(confFilePath)

## Loading sources from file

In [ ]:
sources_hypotesis = """
15.7017e-08 80.3286 1.12047 2.16619 0 2 _2AGLJ2032+4135 0 0 0 0 0.5 5 20 10000 0 100
16.9737e-08 79.3 0.55 2.1 3 2 CYGX3 0 0 0 0 0.5 5 20 10000 0 100
119.303e-08 78.2375 2.12298 1.75823 0 2 _2AGLJ2021+4029 0 1 3307.63 0 0.5 5 20 10000 0 100
"""

sourceFile = "./sources.txt"

with open(sourceFile, "w") as sf:
    sf.write(sources_hypotesis)

In [ ]:
sources = ag.loadSourcesFromFile(sourceFile)

In [ ]:
for s in sources:
    print(s)

## Map generation

In [ ]:
maplistfile = ag.generateMaps()

In [ ]:
!cat $maplistfile

## Estimation of isotropic and galactic background coefficients

In [ ]:
galBkg, isoBkg, calc_bg_maplistfile = ag.calcBkg('CYGX3', galcoeff = [0.7], pastTimeWindow = 14)

In [ ]:
ag.printOptions("model")

In [ ]:
!cat $calc_bg_maplistfile

## Displaying the sky maps

In [ ]:
ag.printOptions("maps")

In [ ]:
ag.displayCtsSkyMaps(smooth=3)

In [ ]:
ag.displayExpSkyMaps()

In [ ]:
ag.displayGasSkyMaps()

## Maximum likelihood analysis

In [ ]:
cygx3 = ag.selectSources('name == "CYGX3"', show=True)

In [ ]:
ag.mle()

In [ ]:
for s in ag.getSources():
    print(s)

In [ ]:
sources = ag.selectSources("sqrtTS > 0")

In [ ]:
if ag.updateSourcePosition("CYGX3"):
    print("Source position updated")


In [ ]:
sources = ag.selectSources("sqrtTS > 0", show=True)

In [ ]:
sources = ag.freeSources('name == "CYGX3"', "pos", False, show=True)

In [ ]:
ag.mle()

In [ ]:
sources = ag.selectSources("sqrtTS > 0", show=True)

In [ ]:
sources = ag.freeSources('name == "CYGX3"', "pos", True, show=True)

In [ ]:
ag.mle()

In [ ]:
sources = ag.selectSources("sqrtTS > 0", show=True)

## Light curve

In [ ]:
lightCurveData = ag.lightCurve("CYGX3", binsize=86400)

In [ ]:
cat $lightCurveData

In [ ]:
ag.displayLightCurve()

## Cleaning up

In [ ]:
ag.deleteAnalysisDir()